## Loading Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

pd.options.display.max_colwidth = 100

In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
pd.options.display.max_colwidth = 100
import re
import os
os.getcwd()

'/Users/sarora48/Documents/Mine/Kaggle-Competitions/Disaster_Tweet_Classfication'

In [3]:
# os.chdir("/kaggle/input/")

## Loading Data and Exploration

In [3]:
train_df = pd.read_csv("nlp-getting-started/train.csv")
test_df = pd.read_csv("nlp-getting-started/test.csv")

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 102.0+ KB


In [7]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


Let's see some tweets where keyword and location are not null.

In [9]:
train_df[~train_df.keyword.isnull() & ~train_df.location.isnull()].head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N,0


Observation
- For row 32, keyword "ablaze" doesn't make sense. Let's go through some tweets for keyword ablaze:

In [8]:
for text in train_df[train_df['keyword']=='ablaze']['text']:
    if 'ablaze' not in text.lower():
        print(text, "\n")

We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw 

Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw 

I gained 3 followers in the last week. You? Know your stats and grow with http://t.co/TIyUliF5c6 

Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw 

First night with retainers in. It's quite weird. Better get used to it; I have to wear them every single night for the next year at least. 

SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintendent Lanford Salmon has r ... - http://t.co/vplR5Hka2u http://t.co/SxHW2TNNLf 

Noches El-Bestia '@Alexis_Sanchez: happy to see my teammates and training hard ?? goodnight gunners.?????? http://t.co/uc4j4jHvGR' 



- For above tweets, there wasn't any keyword 'ablaze' but keyword column indicates otherwise.  We need to keep this in mind while building the model.  
- Now, for more understanding as the saying goes "First, look at your data". So, we will scan through some disastrous and non-disastrous tweets.

In [9]:
print("Disastrous Tweets \n================")
for keyword, text in train_df[train_df['target']==1][['keyword','text']][:20].itertuples(index=False):
    print(keyword,":", text, "\n")

Disastrous Tweets 
nan : Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all 

nan : Forest fire near La Ronge Sask. Canada 

nan : All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected 

nan : 13,000 people receive #wildfires evacuation orders in California  

nan : Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school  

nan : #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires 

nan : #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas 

nan : I'm on top of the hill and I can see a fire in the woods... 

nan : There's an emergency evacuation happening now in the building across the street 

nan : I'm afraid that the tornado is coming to our area... 

nan : Three people died from the heat wave so far 

nan : Haha South Tampa is getting flooded hah- WAIT

Observation:
- "#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi": This tweet doesn't seem to make any sense. The flag is set ablaze but this doesn't seem to be any kind of of disaster.  

Let's look at some non-disastrous tweets.

In [10]:
print("Disastrous Tweets \n================")
for keyword, text in train_df[train_df['target']==0][['keyword','text']][:20].itertuples(index=False):
    print(keyword,":", text, "\n")

Disastrous Tweets 
nan : What's up man? 

nan : I love fruits 

nan : Summer is lovely 

nan : My car is so fast 

nan : What a goooooooaaaaaal!!!!!! 

nan : this is ridiculous.... 

nan : London is cool ;) 

nan : Love skiing 

nan : What a wonderful day! 

nan : LOOOOOOL 

nan : No way...I can't eat that shit 

nan : Was in NYC last week! 

nan : Love my girlfriend 

nan : Cooool :) 

nan : Do you like pasta? 

nan : The end! 

ablaze : We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw 

ablaze : Crying out for more! Set me ablaze 

ablaze : On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N 

ablaze : @PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season. 



In [11]:
# what are the number of unique keywords although they might be misleading at times
train_df.keyword.nunique()

221

## Preprocessing Data

- We don't want any web links as generally they are shortened versions of actual URLs and mostly it will some bit.ly and random characters. So, we will remove them.
- We will store keywords with hastags in hashtags for later use. Then, we will remove those hash symbols from the tweets, so that, for example, #fire and #fires is converted to "fire" only. 

In [36]:
train_df['hastags'] = train_df['text'].apply(lambda t: list(map(lambda x: x.replace("#", ""), 
                                                                re.findall(r"\#\w+", t)))) 
train_df['text'] = train_df.text.replace("#", "").apply(lambda x: re.sub(r"https?://.*", "", x))
train_df.head()

,id,keyword,location,text,target,hastags
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,[earthquake]
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,[]
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,1,[]
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,[wildfires]
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,"[Alaska, wildfires]"


In [66]:
test_df['hastags'] = test_df['text'].apply(lambda t: list(map(lambda x: x.replace("#", ""), 
                                                                re.findall(r"\#\w+", t)))) 
test_df['text'] = test_df.text.replace("#", "").apply(lambda x: re.sub(r"https?://.*", "", x))
test_df.head()

,id,keyword,location,text,hastags
0,0,NaN,NaN,Just happened a terrible car crash,[]
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone.",[earthquake]
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",[]
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,"[Spokane, wildfires]"
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,[]


## Building Model

Let's create a baseline model using old school techniques like tf-idf, and simple machine leanrning algothims. In next kernel, we will try using word embeddings and deep learning for the tweets.  

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(train_df['text'], 
                                                  train_df['target'], 
                                                  test_size=0.2,
                                                  random_state=42, 
                                                  stratify=train_df['target'])

In [79]:
# Basic count of words
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))

count_vect = CountVectorizer(stop_words=stop).fit(X_train)
X_train_counts = count_vect.transform(X_train)
X_train_counts.shape

(6090, 14155)

In [80]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_idf = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tf_idf.transform(X_train_counts)
X_train_tfidf.shape

(6090, 14155)

In [134]:
# creating a function for fine tuning model
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

def model(clf, params, x_tr, y_tr, x_te, y_te, scoring='f1', cv=5, n_jobs=-1):
    if params:
        clf = GridSearchCV(clf, params, scoring, n_jobs=-1, cv=5)
        clf = clf.fit(x_tr, y_tr)
        print("Best Score: ", clf.best_score_)
        print("Best Params:", clf.best_params_)
    else:
        clf = clf.fit(x_tr, y_tr)
        
    y_pred = clf.predict(x_te)
    print("-----------Classification Report---------------")
    print(metrics.classification_report(y_te, y_pred))
    print("-----------Confusion Matrix---------------")
    print(metrics.confusion_matrix(y_te, y_pred))
    print('F1 Socre:', metrics.f1_score(y_te, y_pred))
    return clf

#### Naive Bayes

In [125]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf.fit(X_train_tfidf.todense(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [126]:
# evaluating on the dev set
X_dev_counts = count_vect.transform(X_dev)
X_dev_tf_idf = tf_idf.transform(X_dev_counts)

y_predict = nb_clf.predict(X_dev_tf_idf.todense())

In [127]:
from sklearn.metrics import accuracy_score, f1_score
print('Accuracy:', accuracy_score(y_dev, y_predict))
print('f1_score:', f1_score(y_dev, y_predict))

Accuracy: 0.5975049244911359
f1_score: 0.6227692307692307


#### Logistic Regression

In [130]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
params = {
    'C': [1e-3, 1e-2, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],    
}

log_clf = model(log_clf, params, X_train_tfidf, y_train, X_dev_tf_idf,
               y_dev)

Best Score:  0.7235746071915872
Best Params: {'C': 10, 'penalty': 'l2'}
-----------Classification Report---------------
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       869
           1       0.78      0.74      0.76       654

   micro avg       0.80      0.80      0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523

-----------Confusion Matrix---------------
[[736 133]
 [173 481]]
F1 Socre: 0.7586750788643533


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### SVM Classifier

In [131]:
from sklearn import svm 
params = {'kernel':('linear', 'rbf'), 
              'C':[1, 10]}
svm_clf = svm.SVC()
svm_clf = model(svm_clf, params,  X_train_tfidf, y_train, X_dev_tf_idf,
               y_dev)

Best Score:  0.7270248876906054
Best Params: {'C': 1, 'kernel': 'linear'}
-----------Classification Report---------------
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       869
           1       0.83      0.72      0.77       654

   micro avg       0.82      0.82      0.82      1523
   macro avg       0.82      0.81      0.81      1523
weighted avg       0.82      0.82      0.82      1523

-----------Confusion Matrix---------------
[[775  94]
 [184 470]]
F1 Socre: 0.7717569786535303


#### Random Forest Classifer

In [132]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
params = {
    'n_estimators' : [100, 500],
    'max_features' : ['sqrt', 'log2'],
    'min_samples_split':[10, 20, 30] 
}

rf = model(rf, params, X_train_tfidf, y_train, X_dev_tf_idf, y_dev)

Best Score:  0.7218729493605731
Best Params: {'max_features': 'log2', 'min_samples_split': 20, 'n_estimators': 100}
-----------Classification Report---------------
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       869
           1       0.81      0.72      0.76       654

   micro avg       0.81      0.81      0.81      1523
   macro avg       0.81      0.80      0.80      1523
weighted avg       0.81      0.81      0.81      1523

-----------Confusion Matrix---------------
[[756 113]
 [181 473]]
F1 Socre: 0.7629032258064516


#### XGBoost

In [148]:
import xgboost as xgb
xgdmat_train_tfidf = xgb.DMatrix(X_train_tfidf, y_train)
xgdmat_dev_tfidf = xgb.DMatrix(X_dev_tf_idf, y_dev)

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [147]:
xgb_clf = xgb.XGBClassifier(n_estimators=1000, objective='binary:logistic')
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.005],
    'subsample': [0.7, 0.8, 0.9],
    'min_child_weight': [1,3,5]
}

xgb_clf = model(xgb_clf, params, X_train_tfidf, y_train, X_dev_tf_idf, y_dev)

Best Score:  0.7130185675637901
Best Params: {'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.9}
-----------Classification Report---------------
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       869
           1       0.78      0.71      0.74       654

   micro avg       0.79      0.79      0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523

-----------Confusion Matrix---------------
[[734 135]
 [189 465]]
F1 Socre: 0.7416267942583732


In [150]:
params = {'eta': 0.1, 'seed':0, 'subsample': 0.9, 
          'objective': 'binary:logistic', 'max_depth':7, 
          'min_child_weight':1} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = params, dtrain = xgdmat_train_tfidf, 
                num_boost_round = 3000, nfold = 5,
                metrics = ['f1'], 
                early_stopping_rounds = 50) 

XGBoostError: [00:54:03] src/metric/metric.cc:23: Unknown metric function f1
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000001a25bb4f09 dmlc::LogMessageFatal::~LogMessageFatal() + 57
  [bt] (1) 2   libxgboost.dylib                    0x0000001a25c28c44 xgboost::Metric::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) + 404
  [bt] (2) 3   libxgboost.dylib                    0x0000001a25baf80a xgboost::LearnerImpl::Configure(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > > > > const&) + 842
  [bt] (3) 4   libxgboost.dylib                    0x0000001a

In [ ]:
cv_xgb.tail(5)

In [ ]:
evallist = [(xgdmat_dev_tfidf, 'eval'), (xgdmat_train_tfidf, 'train')]
final_xgb = xgb.train(params, xgdmat_train_tfidf, num_round=, 
                      evallist=evallist)



Next Work:
- Lots of tokens were for numbers like 2 AM. Intuitively, doesn't seem it will contribute a lot. We'll remove numerical features and then test the above models again
- We will try to use keywords/hastags to see how predictive they are. Although it doesn't seem, they will improve upon upon the above model. 
- We will try to use the word embeddings instead of tf-idf featueres. 
- Use deep learning.